In [1]:
import os
from dotenv import load_dotenv
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
import openai
from langchain_groq import ChatGroq
#GROQ_API_KEY = os.environ.get("GROQ_API_KEY", "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")
load_dotenv()

True

In [2]:
llm = ChatGroq(
    model = "llama-3.1-70b-versatile",
    temperature=0.1,
    max_tokens=None,
    timeout=None,
    max_retries=2   
)

In [3]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent
tool=load_tools(["wikipedia"],llm=llm)
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_20448\572361986.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


## Pre-built Tools LangChain


In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=api_wrapper)
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Milvus (vector database)
Summary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.
Milvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.

Page: Retrieval-augmented generation
Summary: Retrieval-Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment infor

## Custom Tools LangChain

In [5]:
from langchain.tools import BaseTool

class calc(BaseTool):
    
    name : str= "Circumference calculator"
    description : str = "use this tool when you need to calculate a circumference using the radius of a circle"

    def _run(self, radius):
        return float(radius)*2.0*3.14

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")

In [6]:
from langchain_groq import ChatGroq

model = ChatGroq(
    model = "llama-3.1-70b-versatile",
    temperature=0.1,
    max_tokens=None,
    timeout=None,
    max_retries=2   
)

messages = [
    ("system", "You are a helpful translator. Translate the usersentence to Kannada."),
    ("human", "I love programming."),
]

model.invoke(messages)

AIMessage(content='ನಾನು ಪ್ರೋಗ್ರಾಮಿಂಗ್ ಅನ್ನು ಪ್ರೀತಿಸುತ್ತೇನೆ.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 53, 'total_tokens': 127, 'completion_time': 0.269090909, 'prompt_time': 0.008840651, 'queue_time': 0.24513799899999997, 'total_time': 0.27793156}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_76dc6cf67d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f05fb13c-2a78-4288-a474-83c88dc4de36-0', usage_metadata={'input_tokens': 53, 'output_tokens': 74, 'total_tokens': 127})

In [7]:
import requests
from langchain.tools import BaseTool

WOLFRAM_ALPHA_URL = 'http://api.wolframalpha.com/v2/query'

client = openai.OpenAI()
math = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': 'You are an expert mathematician who can perform complex math operations with reasoning and proofs.'},
        {'role': 'user', 'content': 'Please answer the query.'}
    ],
    temperature=0.2,
    max_tokens=2048,
    top_p=0.7,
)

def query_wolfram_alpha(query):
    """
    Sends a query to Wolfram Alpha API and returns the response.
    """
    params = {
        'input': query,
        'appid': os.environ.get('WRAM_API_KEY'),
        'format': 'plaintext'
    }
    response = requests.get(WOLFRAM_ALPHA_URL, params=params)
    if response.status_code == 200:
        return response.text
    else:
        return "Error querying Wolfram Alpha."

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def preprocess_query(text):
    
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        model_name='gpt-4o-mini',
        encoding_name='text-embedding-3-small',
        )

    split_text = splitter.split_text(text)
    split_text

    return model.invoke(f'convert {split_text} into wolfram alpha efficient form')

def preprocess_result(text):
    
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        model_name='gpt-4o-mini',
        encoding_name='text-embedding-3-small',
        )

    split_text = splitter.split_text(text)
    split_text

    return model.invoke(f'convert {split_text} into a easily understandable form')


## Prompt Template

In [9]:
template = '''Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}'''

In [10]:
template = PromptTemplate(input_variables=["input", "tools", "agent_scratchpad", "tool_names"], template=template)


# Tool definition

In [11]:
class tool(BaseTool):
    
    name: str = "wolfram_llama_tool"
    description: str = "A tool that integrates Wolfram Alpha and LLaMA model for computational queries."


    def _run(self, query: str):
        """
        Internal run method (used for async execution in LangChain).
        """
        return self.invoke(query)


    def invoke(self, query: str) -> str:
        """
        Custom invoke method for the WolframLlamaTool.
        """
        # Process query with LLaMA (optional, can be used for preprocessing)
        #processed_query = preprocess_query(query)
        
        # Send the processed query to Wolfram Alpha
        wolfram_response = query_wolfram_alpha(query)
        
        # Optionally, process the Wolfram response with LLaMA
        #final_response = preprocess_result(wolfram_response)
        
        return wolfram_response.format(template)
    
cust = tool()
circ = calc()

In [12]:
tools = [
    Tool(
        name="Intermediate Answer",
        func=model,
        description="useful for when you need to ask with search",
        verbose=True
    ),
    Tool(name="wolfram_llama_tool",
         func=cust._run,
         description="A tool that integrates Wolfram Alpha and LLaMA model for computational and theoritical queries.",
         verbose=True
    ),
    Tool(name="Circumference Calculator",
         func=circ._run,
         description="A tool that finds circumference of any circle of known radius",
         verbose=True)
]

In [14]:
agent = create_react_agent(model, tools, prompt=template)
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True,handle_parsing_errors=True)

agent_executor.invoke({
    "input": "Calculate circumference of the circle with radius r = 3.14"
    })

agent_executor.invoke({
    "input": "Integrate e^x2 from 0 to infinity if the function is integrable and also show the step by step process"
    })



> Entering new AgentExecutor chain...
To calculate the circumference of a circle, we need to use the formula C = 2πr, where C is the circumference and r is the radius. Since we have the radius, we can directly use the Circumference Calculator tool.

Action: Circumference Calculator
Action Input: 3.1419.7192Thought: I now know the final answer
Final Answer: The circumference of the circle with radius r = 3.14 is 19.7192.

> Finished chain.


> Entering new AgentExecutor chain...
To integrate $e^{x^2}$ from 0 to infinity, we first need to determine if the function is integrable over this interval. The function $e^{x^2}$ is a Gaussian function, which is known to be integrable over the real line, but the integral does not have an elementary antiderivative. However, we can use the properties of Gaussian functions and special functions to evaluate this integral.

Thought: Since this problem involves integration and special functions, we should use the wolfram_llama_tool to find the integra

{'input': 'Integrate e^x2 from 0 to infinity if the function is integrable and also show the step by step process',
 'output': 'The integral of $e^{x^2}$ from 0 to infinity does not converge, meaning the function is not integrable over this interval.'}